# Imports

In [12]:
import os
from ultralytics import YOLO
from PIL import Image
import numpy as np

# Data Preparation

In [ ]:
# Define source folder
image_folder="D:/Datasets/YOLO_1_Dataset/test/images"
label_folder="D:/Datasets/YOLO_1_Dataset/test/labels"

#find all images and labels
test_images= [f for f in os.listdir(image_folder) if f.endswith((".jpeg", ".JPEG"))]
test_labels= [f for f in os.listdir(label_folder) if f.endswith((".txt"))]
print(f"Gefundene testbilder: {len(test_images)}")
print(f"Gefundene testlabels: {len(test_labels)}")
print(os.path.splitext(test_images[0])[0])


#find image / label pairs
image_bbox_pairs = [
                    (os.path.join(image_folder, img), os.path.join(label_folder,img.replace(".jpeg",".txt").replace(".JPEG",".txt")))
                    for img in  test_images
                    if img.replace(".jpeg",".txt").replace(".JPEG",".txt") in test_labels
]
print(f"Gefundene Paare: {len(image_bbox_pairs)}")


Gefundene testbilder: 5370
Gefundene testlabels: 5370
1.3.6.1.4.1.14519.5.2.1.6279.6001.104507274032170320323347152411
Gefundene Paare: 5370


# Model

In [ ]:
#  function to get the ground truth boundingbox of a image
def load_ground_truth(file_path, image_shape):
    h,w = image_shape[:2]
    boxes=[]

    with open(file_path, "r") as f:
        for line in f.readlines():
            class_id,x_center, y_center, width, height = map(float, line.strip().split())

            x_min = int((x_center - width/2)*w)
            y_min = int((y_center - width/2)*h)
            x_max = int((x_center + width/2)*w)
            y_max = int((y_center + width/2)*h)
        
            boxes.append([x_min,y_min,x_max,y_max])
    return boxes

In [ ]:
# Load model with best weights
model=YOLO("YOLO1/1/weights/best.pt")
print(model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_

In [ ]:
# prediction and ground truth initialization
predictions={}
ground_truths={}

# go throug every image and predict
for image_path, bbox_path in image_bbox_pairs:
    #load image
    image = Image.open(image_path)

    image_np = np.array(image)
    #get ground truth
    gt_boxes = load_ground_truth(bbox_path, image_np.shape)
    ground_truths[os.path.basename(image_path)] = gt_boxes

    #get results of the prediction
    results = model(image_np)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    scores = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()

    predictions[os.path.basename(image_path)]= {
        "boxes": boxes,
        "scores": scores,
        "classes": classes
    }


0: 512x512 (no detections), 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 0.0m

# Results

In [ ]:
# function to keep the prediction with the highest score
def keep_highest_score(predictions):
    new_predictions = {}
    for image_name, data in predictions.items():
        if len(data['scores']) > 0:
            # find index with highgest score
            highest_score_idx = int(np.argmax(data['scores']))
            # keep only highest score bbox
            new_predictions[image_name] = {
                "boxes": data['boxes'][highest_score_idx:highest_score_idx+1],
                "scores": data['scores'][highest_score_idx:highest_score_idx+1],
                "classes": data['classes'][highest_score_idx:highest_score_idx+1]
            }
        else:
            # if no bbox existing keep empty structure
            new_predictions[image_name] = {
                "boxes": np.empty((0, 4), dtype=np.float32),
                "scores": np.empty((0,), dtype=np.float32),
                "classes": np.empty((0,), dtype=np.float32)
            }
    return new_predictions

predictions = keep_highest_score(predictions)

In [11]:
predictions

{'1.3.6.1.4.1.14519.5.2.1.6279.6001.104507274032170320323347152411.jpeg': {'boxes': array([], shape=(0, 4), dtype=float32),
  'scores': array([], dtype=float32),
  'classes': array([], dtype=float32)},
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.104863864417205674007098486529.jpeg': {'boxes': array([], shape=(0, 4), dtype=float32),
  'scores': array([], dtype=float32),
  'classes': array([], dtype=float32)},
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.108828997377593078753121028952.jpeg': {'boxes': array([], shape=(0, 4), dtype=float32),
  'scores': array([], dtype=float32),
  'classes': array([], dtype=float32)},
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.109466173635014704200062503592.jpeg': {'boxes': array([], shape=(0, 4), dtype=float32),
  'scores': array([], dtype=float32),
  'classes': array([], dtype=float32)},
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.109626634592628652877975591041.jpeg': {'boxes': array([], shape=(0, 4), dtype=float32),
  'scores': array([], dtype=float32),
  'classes': array([], dtype=f

In [7]:
ground_truths

{'1.3.6.1.4.1.14519.5.2.1.6279.6001.104507274032170320323347152411.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.104863864417205674007098486529.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.108828997377593078753121028952.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.109466173635014704200062503592.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.109626634592628652877975591041.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.109710522872080626618036223042.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.116069400930896573799773712313.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.116221631153879627914984470663.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.118597469360280284504244918386.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.121898080754492407159861475149.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.138539218439961612704732946102.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.142868044429387263567069942743.jpeg': [],
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.1461657008134969218360078604

In [ ]:
# Calculates the Intersection over Union (IoU) between two bounding boxes
def calculate_iou(box1, box2):
    # Determine the coordinates of the intersection rectangle
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Compute the width and height of the intersection area
    intersection_width = max(0, x2 - x1)
    intersection_height = max(0, y2 - y1)
    intersection_area = intersection_width * intersection_height

    # Compute the area of both bounding boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Compute the union of both bounding boxes
    union_area = box1_area + box2_area - intersection_area

    # Avoid division by zero and return the IoU value
    return intersection_area / union_area if union_area > 0 else 0

def evaluate(predictions, ground_truths, iou_threshold=0.5):

    tp, fp, fn, tn = 0, 0, 0, 0
        
    # Iterate over all predicted images
    for image_id, gt_boxes in ground_truths.items():
        pred_boxes = predictions[image_id]['boxes']
           
        # Case 1: No predictions and no ground truth → True Negative (TN)
        if len(gt_boxes) == 0 and len(pred_boxes) == 0:
            tn += 1
            continue

        matched_gt = set()
        for pred_box in pred_boxes:
            # Berechne IoU mit allen Ground Truths
            ious = [calculate_iou(pred_box, gt_box) for gt_box in gt_boxes]
            max_iou = max(ious) if ious else 0  # maximaler IoU-Wert für diese Vorhersage
            
            if max_iou >= iou_threshold:
                # True Positive: Eine Ground Truth wurde erkannt
                matched_gt.add(ious.index(max_iou))
            for iou in ious:
                if iou >= iou_threshold:
                    # Case 2: Both predictions and ground truth exist
                    tp += 1
                else:
                    # Case 3: Predictions exist but no ground truth → False Positive (FP)
                    fp += 1

        
        # Case 4: No predictions but ground truth exists → False Negative (FN)
        fn += len(gt_boxes) - len(matched_gt)

    # calc metrics
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    accuracy = (tp + tn) / (tp + fp + fn + tn) if (tp + fp + fn + tn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy,
        "specificity": specificity,
        "f1_score": f1_score,
        "tp": tp,
        "fp": fp,
        "fn": fn,
        "tn": tn,
    }


results = evaluate(predictions, ground_truths)
print(results)


{'precision': 0.9247311827956989, 'recall': 0.921315108976727, 'accuracy': 0.9245556764599202, 'specificity': 0.9276807980049875, 'f1_score': 0.9230199851961509, 'tp': 2494, 'fp': 203, 'fn': 213, 'tn': 2604}
